In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime, date, time
#import pyodbc
import requests
from tqdm import tqdm

In [3]:
#sql_extract = pd.read_csv("./data/execution/CC_export__executionsteplog__202309201735.csv", low_memory=False)
#sql_extract = pd.read_csv("./data/execution/_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202309260930.csv", low_memory=False)
sql_extract = pd.read_csv("./data/execution/_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202309281612.csv", low_memory=False)


In [4]:
sql_extract.rename(columns={'linenum': 'lineNum', 'executionid': 'executionLogsId', 'scriptname': 'scriptName', 'label': 'label', 'status': 'step_status', 'starttime': 'startTime', 'endtime': 'endTime', 'duration': 'duration', 'id': 'StepLogsId'}, inplace=True)

In [5]:
sql_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4188860 entries, 0 to 4188859
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   scriptName       object 
 1   StepLogsId       int64  
 2   executionLogsId  int64  
 3   lineNum          int64  
 4   labelkey         object 
 5   label            object 
 6   labelparams      object 
 7   startTime        object 
 8   endTime          object 
 9   duration         float64
 10  step_status      object 
dtypes: float64(1), int64(3), object(7)
memory usage: 351.5+ MB


In [136]:
#sql_extract.head(1000).to_csv("./data/execution/SQL_head.csv", index=False)

In [137]:
raise SystemExit("Initial extraction completed")

SystemExit: Initial extraction completed

/home/wikus/code/ipt3/env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Get EA script history in JSON format via the API

In [ ]:
run_date = '2023-09-18'
extract_start = '2023-08-01T00:00:00.000Z'
extract_finish = '2023-09-20T23:59:00.000Z'

GET /api/bi/scripts?filter[where][createdAt][between][0]=2023-07-09T22:00:00.523Z&filter[where][createdAt][between][1]=2023-09-07T21:59:59.523Z&withHistory=true
url = 'https://ea.executive.automats.app/api/bi/scripts?filter[where][createdAt][between][0]=' + extract_start + 'filter[where][createdAt][between][1]=' + extract_finish + '&withHistory=true'


In [ ]:
custom_headers  = {"Authorization": "aeG09Z8IE54TNw759z6Eg2SAC432WIJ2S4uChfCTXse7xnaUNVmo0vBuoPLYB0o3", "Content-Type": "application/json"}
#url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true'

url = 'https://ea.executive.automats.app/api/bi/scripts?filter[where][createdAt][between][0]=' + extract_start + '&filter[where][createdAt][between][1]=' + extract_finish + '&withHistory=true'
#url = 'https://ea.executive.automats.app/api/bi/scripts?filter[where][startTime][between][0]=' + extract_start + '&filter[where][startTime][between][1]=' + extract_finish + '&withHistory=true'


resp = requests.get(url, headers = custom_headers, verify=False)
print(resp.status_code)

# Previous runtime = 2min 55s

/home/wikus/code/ipt3/env/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea.executive.automats.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


In [ ]:
json_data = json.loads(resp.text)
json_text = json.dumps(json_data, indent=4)

# Previous runtime = 2min 2s

In [ ]:
try:
    os.remove('data/execution/try.json')
except:
    print('File does not exist')

with open('data/execution/try.json', 'w') as fw:
    fw.write(json_text)

### Remove invalid characters from the inbound JSON data

In [ ]:
try:
    os.remove('data/execution/script_history_fixed.json')
except:
    print('File does not exist')

#with open('data/execution/script_history.json', 'r') as fr:
with open('data/execution/try.json', 'r') as fr:    
    # reading line by line
    lines = fr.readlines()
    last_line = len(lines)
    print(len(lines))
    
    for line in tqdm(lines):
        line = re.sub("\u0003", "", line)
        with open('data/execution/script_history_fixed.json', 'a') as fw:
            fw.write(line) 

# Previous runtime = 5m 28s

7403202


100%|██████████| 7403202/7403202 [02:36<00:00, 47258.77it/s]


### Parse the multilevel JSON into a CSV file

In [ ]:
f = open('data/execution/script_history_fixed.json')
data = json.loads(f.read())

In [ ]:
try:
    os.remove('data/execution/check_structure.csv')
except:
    print('File does not exist')
with open('data/execution/check_structure.csv', 'a') as fw:
    #line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId,lastIssues\n'
    line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId\n'
    fw.write(line)
    i = 0
    for level1 in tqdm(data):
        # for key, value in level1.items():
        #     #if i == 100: break
        
                     
        try:
            executionLogs_str = json.dumps(level1['executionLogs'])
            executionLogs_json = json.loads(executionLogs_str)
            for level2 in executionLogs_json:
                for key2, value2 in level2.items():
                    #try:
                    startTime_str = str(level2['startTime'])
                    startTime_str = startTime_str[0:10]
                    startTime = datetime.strptime(startTime_str, '%Y-%m-%d')
                    oldestHistory = datetime.strptime(run_date, '%Y-%m-%d')
                    executionStepLogs_str = json.dumps(level2['executionStepLogs'])
                    executionStepLogs_json = json.loads(executionStepLogs_str)
                    if key2 == 'executionStepLogs' and startTime >= oldestHistory:
                    #if key2 == 'executionStepLogs':

                        topProjectName = str(level1['topProjectName'])
                        name_str = str(level1['name'])
                        # Remove any commas from name 
                        check = (',' in name_str)
                        if check == True:
                            name_str = re.sub(',', ';', name_str)                            
                        sessionId_str = str(level2['sessionId'])
                        id_str = str(level2['id'])
                        scriptId_str = str(level2['scriptId'])
                        scriptName_str = str(level2['scriptName'])
                        # Remove any commas from scriptName 
                        check = (',' in scriptName_str)
                        if check == True:
                            scriptName_str = re.sub(',', ';', scriptName_str)                             
                        state_str = str(level2['state'])
                        status2_str = str(level2['status'])
                        # lastIssues_str = str(level2['lastIssues'])
                        # Remove any commas from lastIssues_str
                        # check = (',' in lastIssues_str)
                        # if check == True:
                        #     lastIssues_str = re.sub(',', '~', lastIssues_str)
                        # check = ('\n' in lastIssues_str)
                        # if check == True:
                        #     lastIssues_str = re.sub('\n', '|', lastIssues_str)                         


                        for level3 in executionStepLogs_json:
                            
                            #for key3, value3 in level3.items():
                            status3_str = str(level3['status'])
                            label_str = str(level3['label'])
                            
                            # These lines replace long text test step labels with short text (that does not wrap)
                            check = ('Click Dynamics 365' in label_str)
                            if check == True:
                                label_str = 'Click Dynamics 365'
                            check = ('Click Sales Hub' in label_str)
                            if check == True:
                                label_str = 'Click Sales Hub'
                            check = ('Type value function eaExecuteVariableValue()' in label_str)
                            if check == True:
                                label_str = 'Type value function eaExecuteVariableValue()'
                            check = ('Select from function eaExecuteVariableValue()' in label_str)
                            if check == True:
                                label_str = 'Select from function eaExecuteVariableValue()'                                    
                            check = ('Select lookup values function eaExecuteVariableValue()' in label_str)
                            if check == True:
                                label_str = 'Select lookup values function eaExecuteVariableValue()'
                            # Remove any commas from label_str 
                            check = (',' in label_str)
                            if check == True:
                                label_str = re.sub(',', ';', label_str)
                            # Now replace all newlines that remain in label_str
                            check = ('\n' in label_str)
                            if check == True:
                                label_str = re.sub('\n', '/nl/', label_str)
                            # Done with replacing text

                            startTime_str = str(level3['startTime'])
                            endTime_str = str(level3['endTime'])
                            duration_str = str(level3['duration'])
                            lineNum_str = str(level3['lineNum'])
                            StepLogs_id_str = str(level3['id'])

                            i = i + 1

                            #line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+','+lastIssues_str+'\n'
                            line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+'\n'
                            fw.write(line)
                            if level3['id'] == 2:
                                print(line)                                
                        # except:
                        #     print('Exception on Level2 for Level1 item '+str(i))
        except:
            #print('Exception for Level1 item '+str(i))
            pass       

    # Previous runtime = 2min 40s   

  0%|          | 0/390 [00:00<?, ?it/s]

100%|██████████| 390/390 [01:02<00:00,  6.27it/s]


### Read the denormalized CSV file into a Pandas dataframe

In [ ]:
df = pd.read_csv('data/execution/check_structure.csv')

# Previous runtime = 2.2s

### Run from here again

In [6]:
rundate = '2023-09-28'
df = sql_extract.copy()

In [7]:
df.rename(columns={'label':'StepLabel'}, inplace=True)

In [8]:
df = df.dropna(subset = ['scriptName']).copy()

In [9]:

df = df.dropna(subset=['startTime']).copy()
df = df.dropna(subset=['endTime']).copy()
df = df.sort_values(by=['startTime'], ascending=False).copy()
df = df.reset_index(drop=True)

In [10]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [11]:
df['duration_sec'] = df.duration/1000
df['duration_mins'] = df.duration/1000/60

In [12]:
#df['date'] = pd.to_datetime(df['startTime'], format='%Y-%m-%d', utc=True).dt.date
df['date'] = pd.to_datetime(df['startTime'], format='mixed', utc=True).dt.date
df['date'] = pd.to_datetime(df['date'])


In [13]:
df = df.query("date == @rundate").copy()  # Remember time is UTC

In [14]:
df['time'] = pd.to_datetime(df['startTime'], utc=True).dt.time

In [15]:
df['time_dec'] = df['time'].astype(str)
df['time_dec'] = df['time_dec'].str[:5]
df['time_dec'] = df['time_dec'].str.replace(':','.')
df['time_dec'] = df['time_dec'].astype(float)

### Add a column to indicate Runs of each test script
As there is no other field available via the API, this field is generated each time the lineNum == 1.  This assumes that there will always be a step 1 in every test case.
Before this can be done, first sort the whole dataframe

In [16]:
df.sort_values(by=['executionLogsId','StepLogsId'], inplace=True)
df = df.reset_index(drop=True)

In [17]:
# This is the new code to allocate a 'run' number and increment it every time the lineNum == 1.  This was generated by ChatGPT based on the code above :-)

df['run'] = (df['lineNum'] == 1).cumsum()

### Generate a field to indicate where test scripts have run to the last step of the script
This assumes that the last step is "End script"
First sort the dataframe by Runs
Then add a new text field that is a concatenation of all the step labels for each run
Finally, check which of those strings contain the text "End script"

In [18]:
runs_np = df.run.unique()
end_script_dict = {}
for i in runs_np:
    end_script_dict[i] = 'Script stopped'
step_error_dict = {}
for i in runs_np:
    step_error_dict[i] = 'No step errors'

In [19]:
df.sort_values(by=['run'], inplace=True)
df = df.reset_index(drop=True)

In [20]:
for idx in df.index:
    #if run == 10: break
    run = df['run'][idx]
    if df['StepLabel'][idx] == 'End script':
        end_script_dict[run] = 'Script completed'
    if df['step_status'][idx] == 'ERROR':
        step_error_dict[run] = 'One or more step errors'

In [21]:
end_script_df = pd.DataFrame.from_dict(end_script_dict,orient ='index',columns=['script_completion'])
end_script_df.reset_index(inplace=True)
end_script_df.rename(columns={'index':'run'}, inplace=True)

In [22]:
step_error_df = pd.DataFrame.from_dict(step_error_dict,orient ='index',columns=['step_error'])
step_error_df.reset_index(inplace=True)
step_error_df.rename(columns={'index':'run'}, inplace=True)

In [23]:
df = pd.merge(
    df,
    end_script_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [24]:
df = pd.merge(
    df,
    step_error_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [25]:
df.rename(columns={'script_status': 'EA_script_status','step_state': 'EA_step_state','step_status': 'EA_step_status','step_error': 'IPT_step_error','script_completion': 'IPT_script_completion'}, inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94971 entries, 0 to 94970
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   scriptName             94971 non-null  object        
 1   StepLogsId             94971 non-null  int64         
 2   executionLogsId        94971 non-null  int64         
 3   lineNum                94971 non-null  int64         
 4   labelkey               23278 non-null  object        
 5   StepLabel              73489 non-null  object        
 6   labelparams            94971 non-null  object        
 7   startTime              94971 non-null  object        
 8   endTime                94971 non-null  object        
 9   duration               94971 non-null  float64       
 10  EA_step_status         94971 non-null  object        
 11  duration_sec           94971 non-null  float64       
 12  duration_mins          94971 non-null  float64       
 13  d

In [27]:
df['KPI'] = 'Not mapped'

In [28]:
# K1
mask = (df['scriptName'].str.contains('Place a sales local order',case=False))
df['KPI'].mask(mask,'K01 Place a sales local order',inplace=True)

# K2
mask = (df['scriptName'].str.contains('Place a B2B local order',case=False))
df['KPI'].mask(mask,'K02 Place a B2B/EDI order and Send Confirmation',inplace=True)

# K3 Place Service Portal Order
# No test script

# K4
mask = (df['scriptName'].str.contains('Amend an existing sales order',case=False))
df['KPI'].mask(mask,'K04 Amend an existing sales order',inplace=True)

# K5
mask = (df['scriptName'].str.contains('Trade Returns Order',case=False))
df['KPI'].mask(mask,'K05 Place a Trade Returns Order',inplace=True)

# K6
mask = (df['scriptName'].str.contains('Submit a B2B Remittance',case=False))
df['KPI'].mask(mask,'K06 B2B Remittance',inplace=True)

# K7 B2B Remittance Adjustment
# No test script

#K8
mask = (df['scriptName'].str.contains('Receipted payment processing',case=False))
df['KPI'].mask(mask,'K08 Receipted payment processing SA Only at the moment',inplace=True)

# K9 aDSD Batch Job posting invoice for HHD billing document 
# No test script

# K10
mask = (df['scriptName'].str.contains('Credit status check',case=False))
df['KPI'].mask(mask,'K10 Credit status check',inplace=True)

# K11
mask = (df['scriptName'].str.contains('Send Load to Roadnet',case=False))
df['KPI'].mask(mask,'K11 Send Load to Roadnet load for Planning',inplace=True)

# K12
mask = (df['scriptName'].str.contains('Receive load from Roadnet',case=False))
df['KPI'].mask(mask,'K12 Receive load from Roadnet into D365',inplace=True)

# K13b
mask = (df['scriptName'].str.contains('Release to warehouse',case=False) & df['scriptName'].str.contains('Roadnet loads',case=False))
df['KPI'].mask(mask,'K13b Release to Warehouse & Complete Picking Work (Roadnet loads)',inplace=True)
# Release to Warehouse (Roadnet loads) & Complete Picking Work & Process OOS

# K13
mask = (df['scriptName'].str.contains('Release to Warehouse',case=False) & df['scriptName'].str.contains('manual',case=False))
df['KPI'].mask(mask,'K13 Release to Warehouse (manually planned loads)',inplace=True)

# K14 Complete Picking Work & Process OOS 
# Not mapped??


# K14b
mask = (df['scriptName'].str.contains('Complete Picking Work',case=False) & df['scriptName'].str.contains('manual loads',case=False))
df['KPI'].mask(mask,'K14b Complete Picking Work & Process OOS (manually planned loads)',inplace=True)

# K15
mask = (df['scriptName'].str.contains('aDSD Load confirmation',case=False))
df['KPI'].mask(mask,'K15 aDSD Load confirmation',inplace=True)

# K16
mask = (df['scriptName'].str.contains('Load upload',case=False) & df['scriptName'].str.contains('Settlement',case=False))
df['KPI'].mask(mask,'K16 Load upload & Settlement',inplace=True)

# K17
mask = (df['scriptName'].str.contains('Create a cost estimate',case=False))
df['KPI'].mask(mask,'K17 Create a cost estimate for all standard costed procured materials',inplace=True)

# K18
mask = (df['scriptName'].str.contains('Imported Statistical',case=False))
df['KPI'].mask(mask,'K18 Import Actual Statistical entries, into Cost Accounting',inplace=True)

# K19
mask = (df['scriptName'].str.contains('Distribute the range of items pending prices',case=False))
df['KPI'].mask(mask,'K19 Distribute the range of items pending prices to another site',inplace=True)

# K20
mask = ((df['scriptName'].str.contains('Cost Rollup',case=False) & df['scriptName'].str.contains('Cost Allocation',case=False)) | df['scriptName'].str.contains('Maintain cost distribution table',case=False))
df['KPI'].mask(mask,'K20 Run Cost Rollup and Cost Allocation Policies',inplace=True)

# K21
mask = (df['scriptName'].str.contains('Place SFA order',case=False))
df['KPI'].mask(mask,'K21 Place SFA order and Send Confirmation',inplace=True)

In [29]:
df.sort_values(by=['KPI', 'scriptName', 'executionLogsId', 'run', 'lineNum'], inplace=True)
df = df.reset_index(drop=True)

In [30]:
df = df.dropna(subset=['StepLabel']).copy()

In [31]:
raise SystemExit("Initial extraction completed")

SystemExit: Initial extraction completed

/home/wikus/code/ipt3/env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Filter the dataset for output on a specific test cycle

In [32]:
exclude_scripts = ['TEST TIMINGS','Test - 10 sessions', 'Gary Test']

In [33]:
df1 = df.query("date == @rundate and time_dec > 10.0 and scriptName not in @exclude_scripts").copy()  # Remember time is UTC
#df1 = df.query("date == @rundate and time_dec > 10.0 and scriptName not in @exclude_scripts").copy()  # Remember time is UTC

In [34]:
df1 = df.reset_index(drop=True).copy()

In [35]:
successful_runs = df1[df1['IPT_script_completion'] == 'Script completed'].groupby('KPI').agg({'run': 'nunique'}).reset_index()


In [36]:
incomplete_runs = df1[df1['IPT_script_completion'] != 'Script completed'].groupby('KPI').agg({'run': 'nunique'}).reset_index()
incomplete_runs.rename(columns={'run': 'Incomplete Runs', 'KPI': 'KPI Process'}, inplace=True)

### Mark those Test Script Steps that must be included in the time measurement

In [37]:
df1['KPI_no'] = ''

In [38]:
df1['Include_in_measure'] = False

In [39]:
kpi_numbers = ['K11', 'K12', 'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20']

In [40]:
for idx in tqdm(df1.index):
    label = df1['StepLabel'][idx]
    prefix = label[0:3]
    if prefix in kpi_numbers:
        if label[3] == 'b':
            df1.at[idx, 'KPI_no'] = prefix + 'b'
        else:
            df1.at[idx, 'KPI_no'] = prefix  
        df1.at[idx, 'Include_in_measure'] = True

100%|██████████| 73489/73489 [00:00<00:00, 234321.88it/s]


In [41]:
filename = 'data/execution/' + rundate +  '_results_filtered.csv'
df1.to_csv(filename,index=False)

In [42]:
#df2 = df1.query("Include_in_measure == True and EA_step_status != 'WARNING' and EA_step_status != 'ERROR'").copy()
df2 = df1.query("Include_in_measure == True and EA_step_status != 'ERROR'").copy()

In [43]:
df2 = df2.drop_duplicates(keep='first')
df2 = df2.reset_index(drop=True)

In [44]:
# 90th Percentile
def q90(x):
    return x.quantile(0.9)


kpi_results = df2.groupby(['KPI_no']).agg({'KPI': 'first', 'duration_sec': ['mean', 'max', q90, 'std'], 'run': 'count'}).reset_index()

# Flatten multi-level columns
kpi_results.columns = ['_'.join(col) for col in kpi_results.columns]
kpi_results = kpi_results.reset_index(drop=True)

In [45]:
kpi_results.rename(columns={'KPI_no_': 'KPI_no', 'KPI_first':'KPI Process', 'duration_sec_mean': 'Average', 'duration_sec_max': 'Max', 'duration_sec_q90': '90th_%', 'duration_sec_std': 'SD', 'run_count': 'Successful Runs'}, inplace=True)

In [46]:
kpi_results = pd.merge(
    kpi_results,
    incomplete_runs,
    how="left",
    on=None,
    left_on='KPI Process',
    right_on='KPI Process',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

kpi_results['Incomplete Runs'] = kpi_results['Incomplete Runs'].fillna(0)

In [47]:
kpi_results = kpi_results.round(decimals=1)

In [48]:
kpi_results

,KPI_no,KPI Process,Average,Max,90th_%,SD,Successful Runs,Incomplete Runs
0,K11,K11 Send Load to Roadnet load for Planning,3.5,11.4,4.5,1.3,91,12
1,K12,K12 Receive load from Roadnet into D365,57.7,74.1,71.1,22.8,6,6
2,K13,K13 Release to Warehouse (manually planned loads),27.3,81.4,48.2,22.9,10,3
3,K14,K13 Release to Warehouse (manually planned loads),8.8,23.1,18.3,6.9,10,3
4,K13b,K13b Release to Warehouse & Complete Picking W...,18.7,61.6,31.9,9.9,258,32
5,K14b,K13b Release to Warehouse & Complete Picking W...,11.5,158.6,24.9,12.8,657,32
6,K15,K15 aDSD Load confirmation,19.8,94.9,35.7,15.4,153,30
7,K18,"K18 Import Actual Statistical entries, into Co...",24.4,24.4,24.4,NaN,1,1


In [49]:
filename = 'data/execution/' + rundate +  '_kpi_results.xlsx'
kpi_results.to_excel(filename,index=False)

In [50]:
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wikus/code/ipt3/env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Insert this to prevent the cells below from trying to write to the sql table
raise SystemExit("Stop right here!")